# Lab 3a: LSTM
In this tutorial we create neural networks using [Long Short-Term Memory (LSTM)](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) cells. The data and its pre-processing for of this notebook is identical to the first lab (Perceptron) to keep the amount of new information limited. Some of the required code-blocks are empty - requiring your imput to complete the model. A few additional questions at the end challenge you to play around with the code and try things for yourselves.

During the session, you will create a LSTM to translate DNA sequences into protein sequences. 
classifies any given DNA sequence as protein coding or not. As a starting point, we use as examples the coding DNA sequences from humans (homo sapiens (HS)). As negatives, we use random sequences of DNA where each nucleotide is drawn from a uniform distribution over the possible nucleotides. We then train a neural network on de [codon frequencies](https://en.wikipedia.org/wiki/DNA_and_RNA_codon_tables) of these sequences.

In addition to human DNA sequences, we also take a look at coding sequences from mice ([mus musculus (MM)](https://en.wikipedia.org/wiki/House_mouse)) and yeast ([saccharomyces cerevisiae (SC)](https://en.wikipedia.org/wiki/Saccharomyces_cerevisiae)). There are subtle differences between the coding frequencies of these species. You will test how well your human-trained model is able to recover the coding sequences for mice and yeast (think of your results in the context of evolutionary distances between species). 


In [1]:

# import pytorch
import torch
import torch.nn as nn
from torch import Tensor
from torch import optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
import lightning as L

# import basic functionality
import random
import numpy as np
import pandas as pd
import itertools

# libraries for plotting
import seaborn as sns
import matplotlib.pyplot as plt

import Bio
from Bio import SeqIO


# Step 1: Pre-processing the data
Here we download and pre-process the dataset. As before, we only consider DNA sequences that are protein coding, contain a integer number of codons, have a start and stop codon, and do not contain any uncertain nucleotides. Finally, we remove duplicates and randomly mix the sequences. Nothing is different from the last time, so you can simply execute these steps and move on to Step 2.

In [ ]:

# download and unpack DNA coding sequences for human, mouse and yeast
############################

!mkdir -p ~/all_seqs
%cd ~/

!wget -P ~/all_seqs/ https://ftp.ensembl.org/pub/current_fasta/homo_sapiens/cds/Homo_sapiens.GRCh38.cds.all.fa.gz
!gzip -df "all_seqs/Homo_sapiens.GRCh38.cds.all.fa.gz"


In [ ]:

# function that loads and processes a FASTA file containing coding sequences
def load_species_cds(file_name):
    train_cds = []
    train_prot = []
    for record in SeqIO.parse(file_name, "fasta"):
        # ensure that sequences are protein coding
        if 'gene_biotype:protein_coding' in record.description:
            if 'transcript_biotype:protein_coding' in record.description:
                if ' cds ' in record.description:
                    if len(record.seq) % 3 == 0:
                        train_cds.append(str(record.seq))
                        train_prot.append(str(record.seq.translate()))
                        
    # keep sequences that are protein coding
    train_cds_filtered = []
    train_prot_filtered = []
    for i in range(len(train_prot)):
        if (train_prot[i][0]=='M') & (train_prot[i][-1]=='*'):
            train_cds_filtered.append(train_cds[i])
            train_prot_filtered.append(train_prot[i])

    # avoid sequences with undetermined/uncertain nucleotides
    train_cds_filtered = [train_cds_filtered[i] for i in range(len(train_cds_filtered)) if ('N' not in train_cds_filtered[i])]
    train_prot_filtered = [train_prot_filtered[i] for i in range(len(train_cds_filtered)) if ('N' not in train_cds_filtered[i])]
 
    # remove duplicates and randomly mix the list of sequences
    seqs = list(zip(train_cds_filtered, train_prot_filtered))
    seqs = list(set(seqs))
    random.shuffle(seqs)
    train_cds_filtered, train_prot_filtered = zip(*seqs)
    
    return list(train_cds_filtered), list(train_prot_filtered)


In [ ]:

# load coding sequences for different species
print('loading human proteins')
#max_len_cds = 100
DNAseq, ProtSeq = load_species_cds("all_seqs/Homo_sapiens.GRCh38.cds.all.fa")

# take a look at some sequences
[ProtSeq[i][0:40]+'...' for i in range(5)]

In [ ]:
print('number of sequences: ', len(ProtSeq))

In [ ]:

nr_test_seqs = 100

# we'll the 100 first sequences as test sequences
test_DNAseq = DNAseq[0:nr_test_seqs]
test_ProtSeq = ProtSeq[0:nr_test_seqs]

# use the remaining sequences as training data
DNAseq = DNAseq[nr_test_seqs:]
ProtSeq = ProtSeq[nr_test_seqs:]


# Step 2: Encoding the sequences
Having prepared the coding sequences and their translation, we convert them into a numeric representation as vectors. To do so, we first construct a language class that stores words of each language and allows us to convert between encoding/indices and words in a language. Next, we define a function that allows us to store any sequence of words (i.e., codons or bases) as a numeric representation. Here we extend every sequence with a start of sentence <SOS> and end of sentence <EOS> token, such that the model knows when to start and stop translating. Finally, we extend every sentence to the same length by padding with an empty "word" that is not translated or used, but allows us to use the identical-length numerical representation of the sentences as input for the model.

In [ ]:
# class to store a language
class Lang:
    # initialize the language, as standard we have start of sentence (SOS), end of sentence (EOS) and a padding to equalize sentence lengths (PAD)
    def __init__(self, name):
        self.name = name
        self.word2index = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>"}
        self.n_words = 3  # Count SOS and EOS

    # function to add sentence to language (add new words in the sentence to our language)
    def addSentence(self, sentence, codon_length):
        for word in [sentence[i:i+codon_length] for i in range(0, len(sentence), codon_length)]:
            self.addWord(word)

    # function to add word to language
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1
            
    # function to convert indices to encodings (i.e., 3 becomes [0,0,0,1,0,0,...])
    def to_encoding(self):
        for key in self.word2index:
            new_val = np.zeros(len(self.word2index),dtype=np.int32)
            new_val[self.word2index[key]] = 1
            self.word2index[key] = new_val
    

# function to encode and pad a sentence
# we use this to take an input sentence and convert it to a sequence of arrays that represent that sentence in a given language
# in the context of proteins, think of this as encoding the bases or codons
def encode_and_pad(language, sentence, codon_length, max_length):
    sos = [language.word2index["<SOS>"]]
    eos = [language.word2index["<EOS>"]]
    pad = [language.word2index["<PAD>"]]

    # split sentence in blocks of a given codon_length
    sentence_split = [sentence[i:i+codon_length] for i in range(0, len(sentence), codon_length)]
        
    # encode sentence in the given language
    sentence_encoded = [language.word2index[word] for word in sentence_split]
    if len(sentence_split) < max_length - 2: 
        # sentence is shorter than max length -2; add SOS and EOS and pad to maximum length
        n_pads = max_length - 2 - len(sentence_split)
        return np.array(sos + sentence_encoded   + eos + pad * n_pads)
    else: 
        # sentence is longer than max length; truncate and add SOS and EOS
        sentence_truncated = sentence_encoded[:max_length - 2]
        return np.array(sos + sentence_truncated + eos)

In [ ]:

# create languages for both the protein and DNA sequences
dna_lang = Lang("dna_seq")
prot_lang = Lang("prot_seq")

# define codon length
codon_length = 3

# add all sequences to the languages
for i in range(len(DNAseq)):
    dna_lang.addSentence(DNAseq[i], codon_length)
    prot_lang.addSentence(ProtSeq[i], 1)

# convert the DNA sequence to an encoding
# that is, we create a vector-representation of the input sequences for the model
dna_lang.to_encoding()
    


As example, we encode here the first six bases and the first 10 amino acids of the first sequence in our data:


In [ ]:
# take a look at the encoding of a DNA sequence
DNAseq[0][0:6], encode_and_pad(dna_lang, DNAseq[0], codon_length, 6)

In [ ]:
# take a look at the encoding of a protein sequence
ProtSeq[0][0:10], encode_and_pad(prot_lang, ProtSeq[0], 1, 10)

In [ ]:
########################
### encode your sequences here
### put the results in the variables 'DNAseq_encoded' and 'ProtSeq_encoded'
### use then encode function defined above
########################

In [ ]:
#take a look at your encoded DNA sequences
DNAseq_encoded[0:3], DNAseq_encoded[0].shape

In [ ]:
#take a look at your encoded protein sequences
ProtSeq_encoded[0:3], ProtSeq_encoded[0].shape

We have now encoded all DNA sequences in the same format: an array where each of the elements encodes one of the possible codons + the SOS, EOS, and PAD elements, each codon in the sequence is then represented by an array (thus, each sequence is now a matrix of (codons in sequence x possible codons). Analogously, for the protein sequences, each element encodes one of the codons in the sequence (by its index from our protein language). Thus, we converted the DNA sequences (variable-length text) into a consistent format (matrices of numbers with fixed length). These variables can now be used as input for a model. Finally, we create the dataloaders for the encoded sequences.

In [ ]:

# define dataloader for the encoded sequences
def get_dataloader(dna_encoded, prot_encoded, batch_size):
    train_x = np.array(dna_encoded)
    train_y = np.array(prot_encoded)
    train_data = TensorDataset(torch.from_numpy(train_x).double(), torch.from_numpy(train_y).double())

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, drop_last=True)
    return train_dataloader


In [ ]:
########################
### create a dataloader for the test and training sequences
### put the results in the variables 'train_dl' and 'val_dl'
### use then dataloader function defined above
### think about batch sizes and the number of training and validation samples
########################

# Step 3: Define model
As a final preparation, we define our [model](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html). 
Create a class that instructs pytorch to make a LSTM model using the given input, hidden size and output parameters. You'll have to define the init and forward functions. Additionally, in the Pytorch Lightning class, you must choose a loss function and optimizer that are appropriate for the problem you are trying to solve. Once you have thought about your model, we will go over the architecture together with the class. Hint: for the LSTM, you'll need to also define the hidden and cell states.

In [43]:

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
########################
### create the model architecture
########################
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyLSTM,self).__init__()
        
        # save input parameters
        # define model layers (LSTM) (pseudocode:)
        self.LSTM = nn.LSTM(input_1, ..., input_k, batch_first = True)
        
    def forward(self,inp):
        inp = inp.to(device)
        
        # define initial hidden and cell states of LSTM
        # run LSTM (pseudocode:)
        output_1, ..., output_n = self.LSTM(input_1, ..., input_m) 
        
        return output
        

In [ ]:
########################
### define the loss function and optimizer
########################

# lightning module to train the sequence model
class SequenceModelLightning(L.LightningModule):
    def __init__(self, input_size, hidden_size, output_size, lr=0.1):
        super().__init__()
        self.model = MyLSTM(input_size, hidden_size, output_size).double()
        self.lr = lr
        # define loss function here (pseudocode:)
        self.loss = nn.MyLoss()

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        input_tensor = batch[0].double()
        target_tensor = batch[1].double()

        output = self.model(input_tensor)
        loss = self.loss(output.view(-1, output.shape[2]),target_tensor.view(-1).long())
        self.log("train_loss", loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        input_tensor = batch[0].double()
        target_tensor = batch[1].double()

        output = self.model(input_tensor)
        loss = self.loss(output.view(-1, output.shape[2]),target_tensor.view(-1).long())
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        # define optimizer here
    

In [ ]:
########################
### define the input parameters for the training loop
########################

# define the model and training loop
lit_model = SequenceModelLightning(input_size = ...,
                                  hidden_size = ...,
                                  output_size = ...,
                                  lr = ...)

# define the trainer
trainer = L.Trainer(devices = 1, 
                    max_epochs = ...)

# learn the weights of the model
trainer.fit(lit_model, train_dl, val_dl)


In [ ]:

# performance of model on validation data
trainer.validate(lit_model, val_dl)


In [ ]:
# print model architecture
mmm = lit_model.model
mmm

In [ ]:
#%reload_ext tensorboard
#%tensorboard --logdir=lightning_logs/

# Step 4: Test the model
Finally, we test the model on a random sequence

In [ ]:
########################
### encode your test sequences here
### put the results in the variables 'test_DNAseq_encoded' and 'test_ProtSeq_encoded'
### use then encode function defined before
########################

In [ ]:
# pick a random sequence
random_pair = np.random.randint(0,100)

# send input sequence to device
input_tensor = torch.tensor([test_DNAseq_encoded[random_pair]]).double()
protein_translation = test_ProtSeq[random_pair]

# compute translation of sequence
output = mmm(input_tensor)

# convert output back to protein sequence by taking the most likely amino acid per position and skipping SOS, EOS and PAD
result = "".join([prot_lang.index2word[i] for i in output.cpu().topk(1)[1].view(-1).numpy()]).split('<PAD><PAD><PAD>')[0]# if i not in [key for key in Lang('').index2word]])

# print sequences
min_len = np.min([len(result),len(protein_translation)])
print('     '+protein_translation)
print(result,end='\n\n')

# print accuracy
result = "".join([prot_lang.index2word[i] for i in output.cpu().topk(1)[1].view(-1).numpy() if i not in [key for key in Lang('').index2word]])
print('Accuracy of aa calling over the sequence: ', np.sum([protein_translation[i] == result[i] for i in range(min_len)])/min_len)


# Step 5: Questions
1) Encode the training and test sequences
2) Define a LSTM and loss function
3) Can you achieve a perfect accuracy on arbitrary-length sequences?
4) What is the minimum model size to reach a 'good' accuracy?
4) Can you train the model and translate DNA without using the SOS / EOS tokens?
5) Can you modify the code to train it on a single DNA sequence, and achieve reasonable accuracy?